## This notebook will show my code for the project

In [13]:
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import numpy as np
#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [3]:
# read data
df = pd.read_csv('Amsterdam.csv')
df.drop(['Unnamed: 0'], axis = 1, inplace = True)
df.rename(columns = {'0' : 'Latitude', '1' : 'Longitude', 'Neighborhoods' : 'Neighborhood' }, inplace = True)
print(df.head())

  Borough           Neighborhood   Latitude  Longitude
0    West  Frederik Hendrikbuurt  52.376956   4.874085
1    West              Houthaven  52.393376   4.881680
2    West      Spaarndammerbuurt  52.389663   4.879369
3    West      Staatsliedenbuurt  52.380286   4.870951
4    West         Zeeheldenbuurt  52.389330   4.888242


In [4]:
# plot map amsterdam with sampling points are the neighborhoods 
# latitude and longitude of Amsterdam to create map 
latitude = 52.3545362
longitude = 4.7638781

# create map of Amsterdam using latitude and longitude values
map_amsterdam = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_amsterdam)  
#map_amsterdam

In [5]:
# Define foursquare API credential to use API
CLIENT_ID = 'MC2SRBWTQ4OJXCCA1KUMROH2SOVUVP103QMRIRDMPJHEPGNK' # Foursquare ID
CLIENT_SECRET = 'IFMJISV3YMKFQI1WTGAUAKTKPQ3EUPBOO4J2BQ55Z1M1WRQY' # Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MC2SRBWTQ4OJXCCA1KUMROH2SOVUVP103QMRIRDMPJHEPGNK
CLIENT_SECRET:IFMJISV3YMKFQI1WTGAUAKTKPQ3EUPBOO4J2BQ55Z1M1WRQY


In [6]:
# function to return nearby venues for any latitude and longitude
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
# function to sort the venues in descending order.
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [7]:
# create data for each Borough
center_data = df[df['Borough'] == 'Centrum (Centre)'].reset_index(drop=True)
west_data = df[df['Borough'] == 'West'].reset_index(drop=True)
south_data = df[df['Borough'] == 'Zuid (South)'].reset_index(drop=True)
east_data = df[df['Borough'] == 'Oost (East)'].reset_index(drop=True)
newwest_data = df[df['Borough'] == 'Nieuw-West(New West)'].reset_index(drop=True)
southeast_data = df[df['Borough'] == 'Zuidoost(Southeast)'].reset_index(drop=True)
North_data = df[df['Borough'] == 'Noord (North)'].reset_index(drop=True)
westgateway_data = df[df['Borough'] == 'Westpoort(West Gateway)'].reset_index(drop=True)

In [8]:
# set limit venues and radius to explore
LIMIT = 150
radius = 600

## 1. Get venues and segmentation in center

In [9]:
center_venues = getNearbyVenues(names=center_data['Neighborhood'],
                                   latitudes=center_data['Latitude'],
                                   longitudes=center_data['Longitude']
                                  )

Binnenstad
Grachtengordel
Haarlemmerbuurt
Jodenbuurt
Jordaan
Kadijken
Lastage
Oosterdokseiland
Oostelijke Eilanden
Plantage
Rapenburg
Uilenburg
Westelijke Eilanden
Weteringschans


In [10]:
# check venues data frame and compute number of unique features 
print('* size of the resulting dataframe', center_venues.shape)
print(center_venues.head())
print('* how many venues were returned for each neighborhood', center_venues.groupby('Neighborhood').count())
print('* There are {} uniques categories.'.format(len(center_venues['Venue Category'].unique())))

* size of the resulting dataframe (1109, 7)
  Neighborhood  Neighborhood Latitude  Neighborhood Longitude          Venue  \
0   Binnenstad              52.373128                 4.88808  De Bierkoning   
1   Binnenstad              52.373128                 4.88808     The Hoxton   
2   Binnenstad              52.373128                 4.88808    The Duchess   
3   Binnenstad              52.373128                 4.88808    W Amsterdam   
4   Binnenstad              52.373128                 4.88808    Hummingbird   

   Venue Latitude  Venue Longitude     Venue Category  
0       52.372404         4.889795         Beer Store  
1       52.371863         4.887487              Hotel  
2       52.372712         4.889253  French Restaurant  
3       52.372787         4.890006              Hotel  
4       52.371641         4.889550        Coffee Shop  
* how many venues were returned for each neighborhood                      Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighbor

In [11]:
# one hot encoder for venue data frames
# one hot encoding
center_onehot = pd.get_dummies(center_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
center_onehot['Neighborhood'] = center_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [center_onehot.columns[-1]] + list(center_onehot.columns[:-1])
center_onehot = center_onehot[fixed_columns]

#print(center_onehot.head())
print(center_onehot.shape)

# group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
center_grouped = center_onehot.groupby('Neighborhood').mean().reset_index()
center_grouped

(1109, 183)


,Neighborhood,Zoo Exhibit,Afghan Restaurant,African Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bagel Shop,Bakery,Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bistro,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bridge,Buffet,Burger Joint,Burrito Place,Bus Stop,Butcher,Café,Camera Store,Canal,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Creperie,Cruise,Cupcake Shop,Deli / Bodega,Design Studio,Dessert Shop,Diner,Dive Bar,Dutch Restaurant,Electronics Store,Exhibit,Fish & Chips Shop,Fish Market,Flea Market,Food & Drink Shop,French Restaurant,Furniture / Home Store,Garden,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Historic Site,History Museum,Hostel,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Latin American Restaurant,Lebanese Restaurant,Liquor Store,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Movie Theater,Multiplex,Museum,Music Venue,Office,Opera House,Optical Shop,Organic Grocery,Outdoor Sculpture,Palace,Park,Pastry Shop,Performing Arts Venue,Pet Store,Pier,Pizza Place,Planetarium,Plaza,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shoe Store,Shopping Mall,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,South American Restaurant,Souvenir Shop,Spa,Spanish Restaurant,Stables,Steakhouse,Supermarket,Sushi Restaurant,Swiss Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Tram Station,Turkish Restaurant,Udon Restaurant,Vacation Rental,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Whisky Bar,Windmill,Wine Bar,Women's Store,Yoga Studio,Zoo
0,Binnenstad,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.020000,0.060000,0.000000,0.020000,0.00,0.01,0.000000,0.000000,0.000000,0.030000,0.000000,0.02,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.070000,0.000000,0.010000,0.00,0.010000,0.010000,0.01,0.000000,0.000000,0.02,0.020000,0.020000,0.00,0.000000,0.000000,0.000000,0.01,0.000000,0.010000,0.00,0.00,0.020000,0.00,0.030000,0.000000,0.00,0.010000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.050000,0.02,0.00,0.010000,0.010000,0.000000,0.00,0.02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.010000,0.000000,0.100000,0.020000,0.000000,0.010000,0.000000,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.01,0.00,0.00,0.000000,0.010000,0.000000,0.030000,0.000000,0.01,0.000000,0.000000,0.000000,0.01,0.000000,0.00,0.00,0.00,0.010000,0.000000,0.00,0.000000,0.000000,0.010000,0.00,0.01,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,0.00,0.000000,0.000000,0.010000,0.00,0.000000,0.01,0.000000,0.000000,0.020000,0.000000,0.00,0.020000,0.000000,0.000000,0.010000,0.000000,0.01,0.010000,0.00,0.010000,0.000000,0.000000,0.01,0.000000,0.000000,0.00,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.020000,0.00,0.010000,0.000000,0.00,0.01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
1,Grachtengordel,0.000000,0.00,0.000000,0.000000,0.00

In [12]:
# create the new dataframe and display the top 10 venues for each neighborhood
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = center_grouped['Neighborhood']

for ind in np.arange(center_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(center_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

NameError: name 'np' is not defined

In [2]:
# segmentation
# set number of clusters
kclusters = 3

center_grouped_clustering = center_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(center_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

NameError: name 'center_grouped' is not defined

In [51]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

center_merged = center_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
center_merged = center_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

center_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Centrum (Centre),Binnenstad,52.373128,4.888080,0,Hotel,Café,Bar,French Restaurant,Marijuana Dispensary,Dessert Shop,Bookstore,Italian Restaurant,Plaza,Steakhouse
1,Centrum (Centre),Grachtengordel,52.370837,4.885478,0,Hotel,Bar,Restaurant,Cheese Shop,French Restaurant,Bookstore,Chocolate Shop,Café,Coffee Shop,Sandwich Place
2,Centrum (Centre),Haarlemmerbuurt,52.382441,4.887193,0,Bar,Italian Restaurant,Café,Marijuana Dispensary,Restaurant,Deli / Bodega,Yoga Studio,Sandwich Place,Organic Grocery,Tapas Restaurant
3,Centrum (Centre),Jodenbuurt,52.368870,4.900223,2,Marijuana Dispensary,Hotel,Bar,Coffee Shop,Italian Restaurant,Theater,Café,Bagel Shop,Brewery,History Museum
4,Centrum (Centre),Jordaan,52.375416,4.881096,0,Bar,Café,Coffee Shop,Hotel,Furniture / Home Store,Indonesian Restaurant,Thai Restaurant,Museum,Pizza Place,Record Shop


In [53]:
# longitude and latitude of center of amsterdam
latitude = 52.3733713
longitude = 4.8689007

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(center_merged['Latitude'], center_merged['Longitude'], center_merged['Neighborhood'], center_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
#map_clusters

In [54]:
# examine each classified group
center_merged.loc[center_merged['Cluster Labels'] == 0, center_merged.columns[[1] + list(range(5, center_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Binnenstad,Hotel,Café,Bar,French Restaurant,Marijuana Dispensary,Dessert Shop,Bookstore,Italian Restaurant,Plaza,Steakhouse
1,Grachtengordel,Hotel,Bar,Restaurant,Cheese Shop,French Restaurant,Bookstore,Chocolate Shop,Café,Coffee Shop,Sandwich Place
2,Haarlemmerbuurt,Bar,Italian Restaurant,Café,Marijuana Dispensary,Restaurant,Deli / Bodega,Yoga Studio,Sandwich Place,Organic Grocery,Tapas Restaurant
4,Jordaan,Bar,Café,Coffee Shop,Hotel,Furniture / Home Store,Indonesian Restaurant,Thai Restaurant,Museum,Pizza Place,Record Shop
9,Plantage,Zoo Exhibit,Bar,Hotel,Café,Pizza Place,Breakfast Spot,Restaurant,Italian Restaurant,Museum,History Museum
12,Westelijke Eilanden,Coffee Shop,Bar,Café,Chinese Restaurant,Sandwich Place,Restaurant,Yoga Studio,Italian Restaurant,Seafood Restaurant,Burrito Place
13,Weteringschans,Bar,Café,Italian Restaurant,Art Museum,Coffee Shop,Hotel,Museum,Restaurant,Gastropub,Steakhouse


In [55]:
# examine each classified group
center_merged.loc[center_merged['Cluster Labels'] == 1, center_merged.columns[[1] + list(range(5, center_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
5,Kadijken,Zoo Exhibit,Restaurant,Plaza,Supermarket,Italian Restaurant,Seafood Restaurant,Planetarium,Park,Organic Grocery,Museum


In [56]:
# examine each classified group
center_merged.loc[center_merged['Cluster Labels'] == 2, center_merged.columns[[1] + list(range(5, center_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Jodenbuurt,Marijuana Dispensary,Hotel,Bar,Coffee Shop,Italian Restaurant,Theater,Café,Bagel Shop,Brewery,History Museum
6,Lastage,Hotel,Bar,Hostel,Chinese Restaurant,Marijuana Dispensary,Italian Restaurant,History Museum,Grocery Store,Pizza Place,Café
7,Oosterdokseiland,Hotel,Restaurant,Hostel,Bar,Hotel Bar,Pub,Italian Restaurant,Thai Restaurant,Steakhouse,French Restaurant
10,Rapenburg,Bar,Hotel,Hostel,History Museum,Breakfast Spot,Coffee Shop,Pizza Place,Italian Restaurant,Harbor / Marina,Chinese Restaurant
11,Uilenburg,Bar,Hotel,Italian Restaurant,Hostel,Marijuana Dispensary,History Museum,Chinese Restaurant,Pizza Place,Coffee Shop,Bagel Shop


In [57]:
# examine each classified group
center_merged.loc[center_merged['Cluster Labels'] == 3, center_merged.columns[[1] + list(range(5, center_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Oostelijke Eilanden,Bus Stop,Café,Park,Gym / Fitness Center,Mediterranean Restaurant,Spa,Breakfast Spot,Snack Place,Brewery,Seafood Restaurant
